# Single image analysis

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [189]:
def resize_to_window(frame):
    frame = cv.resize(frame,(frame.shape[0]//3,frame.shape[1]//3))
    return frame

def extract_pit_contours(frame):
    """ Detect and calculate area for each pit. Thresh old of minimum detection is set to 100 pixel.
    return, the image with each pit highlighted in green with its bonding box. Also returns, it coordinates and the
    area calculated at each frame.
        => Allows the seperation between pits having a growing area (stable) and pits that have a stable area (unstable)
    """
    gray = cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(gray,(3,3),0)
    _, thresh = cv.threshold(blur, 80, 255, cv.THRESH_BINARY)
    kernel = np.ones((21,21),np.uint8)
    opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel)
    equ = cv.equalizeHist(blur)


    contours,_ = cv.findContours(opening, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    pit_roi = []
    areas= []
    for cnt in contours:
        area = cv.contourArea(cnt)
        if 1000000 > area >= 100: 
            cv.drawContours(frame,[cnt],-1,(0,255,0),2)
            x,y,w,h = cv.boundingRect(cnt)
            pit_roi.append((x,y,w,h))
            areas.append(area)
            cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            cv.putText(frame,f"{area:.0f}",(x,y-30),
                       cv.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)


    cv.imshow("image", resize_to_window(frame))
    return frame, pit_roi, areas

def reference_frame(pits_dict):
    """
    found the reference frame at which all the pits detected during all the video are visible
    """
    ref_frame = None
    last_areas = pits_dict["area"][len(pits_dict["frame"])-10]
    average_nbr_of_objects = round(np.mean([len(objct) for objct in pits_dict["area"]]))+1
    for frame,objct in enumerate(pits_dict["area"]):
        if len(objct) == average_nbr_of_objects:
            ref_frame = frame
            break
    return ref_frame

def idx_corresponding_pit(ref_rois,end_rois):
    """
    Variables :
        -  ref_rois = array corresponding to the rois of the first frame that has a len corresponding to the mean len observed for all frames
        -  end_rois = array corresponding to the roi at the end of the 
    Out put :
    Gives an array of the rois and their index on the end_rois corresponding to the pit that where detected at the ref rois

    Purpose : 
    This function is mandatory in order to recover pits from unwanted objects that where detected by the algorythme.
    """
        
    idx = []
    
    for roi_ref in ref_rois:
        x_ref = roi_ref[0]
        y_ref = roi_ref[1]

        delta_x = [np.abs(x_ref-x_end[0]) for x_end in end_rois]
        delta_y = [np.abs(x_ref-x_end[1]) for x_end in end_rois]

        min_x_idx = np.where(delta_x==np.min(delta_x))
        min_y_idx = np.where(delta_y==np.min(delta_y))

        if min_y_idx == min_x_idx :
            idx.append(min_y_idx)
        else :
            print(min_x_idx,min_y_idx)

    return idx
            

        

def pit_area_analysis(video_paths="Videos/30mM.mp4"):
    """
    Labelized each pit and associate to it a value between 1 (stable pit) and 2 (unstable pit).
    returns a dictionnary storing the each frame with the coordinated of each pit and the size of each bonding box.
    Also return the label associated with each detected objects.

    INPUT : video_paths of the corrosion video
    OUTPUT : frame, (x,y,w,h),area,label with,
    
         - frame : numpy array corresponding to the image at a particular frame
         - (x,y,w,h) : numpy array giving coordinates for each objetcs. (x,y) horizontal and vertical coordinates , (w,h) width and height of the bonding box
         - area : numpy array of the same size as (x,y,w,h) with the area of each object
         - label : numpy array of the same size as (x,y,w,h) with the label (1: stable pit or  2: unstable pit) for each object
    """
    cap = cv.VideoCapture(video_paths)
    
    pit_coordinates = {"frame":[],"(x,y,w,h)":[],"area":[],"label":[]}
    frame_number = 0
    total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_number += 1
    
        displayed_img = frame.copy()
        img,roi,area = extract_pit_contours(displayed_img)
        if frame_number == total_frames - 10:
            pit_coordinates["frame"].append(img)
        pit_coordinates["(x,y,w,h)"].append(roi)
        pit_coordinates["area"].append(area)
    
        key = cv.waitKey(25) & 0xFF
        if frame_number == 228:
            break
        elif key == ord('q'):
            break
    
    
    cap.release()
    cv.destroyAllWindows()

    return pit_coordinates

def pit_labelling(pit_coordinates):

    #Define the average number of objects observed to only keep the object that are consistantly observed all over the video
    average_nbr_of_objects = round(np.mean([len(objct) for objct in pit_coordinates["area"]]))+1 
    
    #Calculate the last frame having a number of object equal to the average nbr of objects all over the video and the areas associated 
    #with that frame
    ref_frame = reference_frame(pit_coordinates)
    last_areas = None
    last_areas_frame = None
    ref_array = pit_coordinates['area'][ref_frame]
    for frame, areas in enumerate(pit_coordinates["area"][::-1]):
        if len(areas) == len(ref_array):
            last_areas = areas
            last_areas_frame = len(pit_coordinates["area"])- frame
            break
    print(ref_frame)
    
    # Generate an array the same size as the reference array with labels to each objects
    labels = np.zeros_like(ref_array)
    for i,roi in enumerate(pit_coordinates['(x,y,w,h)'][ref_frame]):
        print(i,roi_tracker(roi,pit_coordinates['(x,y,w,h)'][last_areas_frame-1]))
        ref_area = pit_coordinates['area'][ref_frame][i]
        end_area = pit_coordinates['area'][last_areas_frame-1][roi_tracker(roi,pit_coordinates['(x,y,w,h)'][last_areas_frame-1])]
        print(f"END AREA : {end_area} REF ARE {ref_area}")
        print(f"Delta {end_area-ref_area}")
        if end_area-ref_area > 3000:
            labels[i] = 1.0 
        else:
            labels[i] = 2.0
    pit_coordinates["label"].append(labels)
    print(pit_coordinates["label"])



    # wright on a frame the type of objects observed based on the label array
    labeled_img = pit_coordinates['frame'][0]
    for roi,label in zip(pit_coordinates['(x,y,w,h)'][ref_frame],pit_coordinates["label"][-1]):
        x,y = roi[0],roi[1]
        if label == 1:
            cv.putText(labeled_img,"STABLE",(x,y+15),cv.FONT_HERSHEY_SIMPLEX,2,(0,0,0),2)
        elif label ==2:
            cv.putText(labeled_img,"UNSTABLE",(x,y),cv.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)

        for roi_ref,roi_end in zip(pit_coordinates['(x,y,w,h)'][reference_frame(pit_coordinates)],pit_coordinates["(x,y,w,h)"][last_areas_frame-1]):
            x_ref,y_ref = roi_ref[0],roi_ref[1]
            x_end,y_end = roi_end[0],roi_end[1]
                  


    
    cv.imshow("labeled_img",resize_to_window(labeled_img))
    cv.waitKey(0)
    cv.destroyAllWindows()

    return

# Video analysis

In [190]:
pit_coordinates = pit_area_analysis("Videos/50mM.mp4")
dict_ = pit_coordinates.copy()

In [188]:
pit_labelling(dict_)

228
0 0
END AREA : 141.5 REF ARE 138.0
Delta 3.5
1 1
END AREA : 2430.0 REF ARE 2363.0
Delta 67.0
2 3
END AREA : 20590.0 REF ARE 8924.0
Delta 11666.0
3 4
END AREA : 279.0 REF ARE 273.5
Delta 5.5
4 5
END AREA : 273.0 REF ARE 261.5
Delta 11.5
5 6
END AREA : 164.0 REF ARE 163.0
Delta 1.0
6 7
END AREA : 2737.5 REF ARE 108.0
Delta 2629.5
7 7
END AREA : 2737.5 REF ARE 4085.0
Delta -1347.5
8 8
END AREA : 254.5 REF ARE 248.5
Delta 6.0
9 9
END AREA : 570.5 REF ARE 561.5
Delta 9.0
10 10
END AREA : 8986.0 REF ARE 8141.0
Delta 845.0
11 11
END AREA : 133.0 REF ARE 132.5
Delta 0.5
12 12
END AREA : 263.0 REF ARE 259.0
Delta 4.0
13 13
END AREA : 329.0 REF ARE 319.5
Delta 9.5
14 14
END AREA : 2847.5 REF ARE 2718.0
Delta 129.5
15 15
END AREA : 564.0 REF ARE 553.5
Delta 10.5
16 16
END AREA : 254.5 REF ARE 255.5
Delta -1.0
17 17
END AREA : 330.5 REF ARE 326.5
Delta 4.0
18 18
END AREA : 435.5 REF ARE 431.0
Delta 4.5
19 19
END AREA : 224.0 REF ARE 221.5
Delta 2.5
20 20
END AREA : 111.0 REF ARE 102.0
Delta 9.

## Labelling

In [18]:
cap = cv.VideoCapture("Videos/50mM.mp4")
frame_number = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    
    for idx, roi in enumerate(pit_coordinates['(x,y,w,h)'][frame_number]):
        x = roi[0]
        y = roi[1]
        cv.circle(frame,(x,y),12,(255,0,0),-1)
        cv.putText(frame,f"{idx}",(x,y),cv.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)

    cv.imshow("frame",resize_to_window(frame))
    frame_number += 1
    
    key = cv.waitKey(25) & 0xFF
    if key == ord('q'):
        break


cap.release()
cv.destroyAllWindows()

In [129]:
def roi_tracker(roi_ref,roi_array_end):
    """ 
    Input : arrays of reference and end frame
    Output : index in the end roi array for each roi of the reference array
    """

    delta = [np.abs(roi_ref[0]-roi_end[0]) + np.abs(roi_ref[1]-roi_end[1]) for roi_end in roi_array_end]
    min_ = min(delta)
    min_idx = np.where(delta == min_)


    return min_idx[0][0]
    
    

In [70]:
print(pit_coordinates['(x,y,w,h)'][216])
print(pit_coordinates['(x,y,w,h)'][612])

first_216 = pit_coordinates['(x,y,w,h)'][216][2]

delta = [np.abs(first_216[0]-roi[0]) + np.abs(first_216[1]-roi[1]) for roi in pit_coordinates['(x,y,w,h)'][612]]


min_ = min(delta)

print(min_)

print(np.where(delta == min_))

print(pit_coordinates['area'][216][3],pit_coordinates['area'][612][3])



[(539, 2175, 14, 14), (570, 2139, 61, 53), (1833, 1857, 114, 122), (941, 1797, 20, 20), (1954, 1662, 20, 19), (652, 1455, 16, 15), (1343, 1425, 87, 81), (310, 1355, 24, 18), (551, 1130, 29, 29), (791, 936, 129, 116), (1112, 883, 15, 14), (796, 831, 21, 19), (1791, 592, 21, 21), (1253, 459, 67, 67), (1847, 421, 27, 29), (832, 377, 18, 20), (175, 318, 22, 22), (1439, 150, 25, 25), (724, 133, 17, 19), (1879, 107, 12, 12), (1470, 101, 52, 47), (378, 75, 22, 19)]
[(539, 2174, 14, 15), (570, 2138, 61, 54), (1958, 1817, 13, 12), (1793, 1816, 165, 194), (942, 1796, 20, 21), (1955, 1661, 20, 20), (652, 1454, 16, 16), (1357, 1426, 70, 83), (310, 1355, 25, 18), (552, 1130, 29, 29), (791, 911, 150, 140), (1112, 882, 15, 14), (797, 831, 20, 19), (1791, 591, 22, 22), (1254, 458, 67, 67), (1847, 421, 28, 29), (833, 377, 18, 20), (175, 318, 22, 21), (1439, 149, 26, 25), (724, 133, 17, 19), (1879, 106, 13, 13), (1471, 100, 51, 47), (378, 74, 22, 20)]
81
(array([3], dtype=int64),)
275.0 21126.0
